#### This notebook demonstrates the use of adversarial debiasing algorithm to learn a fair classifier.
Adversarial debiasing [1] is an in-processing technique that learns a classifier to maximize prediction accuracy and simultaneously reduce an adversary's ability to determine the protected attribute from the predictions. This approach leads to a fair classifier as the predictions cannot carry any group discrimination information that the adversary can exploit. We will see how to use this algorithm for learning models with and without fairness constraints and apply them on the Adult dataset.

In [1]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult, load_preproc_data_compas, load_preproc_data_german

from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt

import random
random.seed(0)
import numpy as np
np.random.seed(0)
import tensorflow as tf
tf.random.set_seed(0)

#### Load dataset and set options

In [2]:
# Get the dataset and split into train and test
dataset_orig = load_preproc_data_adult()

privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

In [3]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(34189, 18)


#### Favorable and unfavorable labels

1.0 0.0


#### Protected attribute names

['sex', 'race']


#### Privileged and unprivileged protected attribute values

[array([1.]), array([1.])] [array([0.]), array([0.])]


#### Dataset feature names

['race', 'sex', 'Age (decade)=10', 'Age (decade)=20', 'Age (decade)=30', 'Age (decade)=40', 'Age (decade)=50', 'Age (decade)=60', 'Age (decade)=>=70', 'Education Years=6', 'Education Years=7', 'Education Years=8', 'Education Years=9', 'Education Years=10', 'Education Years=11', 'Education Years=12', 'Education Years=<6', 'Education Years=>12']


#### Metric for original training data

In [4]:
# Metric for the original dataset
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())
metric_orig_test = BinaryLabelDatasetMetric(dataset_orig_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())

#### Original training dataset

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.193075
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.198048


In [5]:
min_max_scaler = MaxAbsScaler()
dataset_orig_train.features = min_max_scaler.fit_transform(dataset_orig_train.features)
dataset_orig_test.features = min_max_scaler.transform(dataset_orig_test.features)
metric_scaled_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
display(Markdown("#### Scaled dataset - Verify that the scaling does not affect the group label statistics"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_train.mean_difference())
metric_scaled_test = BinaryLabelDatasetMetric(dataset_orig_test, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_test.mean_difference())


#### Scaled dataset - Verify that the scaling does not affect the group label statistics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.193075
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.198048


### Learn plan classifier without debiasing

In [6]:
# Load post-processing algorithm that equalizes the odds
# Learn parameters with debias set to False
plain_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False)

In [7]:
plain_model.fit(dataset_orig_train)

(Training Classifier) epoch 0; iter: 0; batch classifier loss: 128.159821
(Training Classifier) epoch 1; iter: 0; batch classifier loss: 61.377651
(Training Classifier) epoch 2; iter: 0; batch classifier loss: 17.932432
(Training Classifier) epoch 3; iter: 0; batch classifier loss: 1.169655
(Training Classifier) epoch 4; iter: 0; batch classifier loss: 0.925180
(Training Classifier) epoch 5; iter: 0; batch classifier loss: 0.855062
(Training Classifier) epoch 6; iter: 0; batch classifier loss: 0.823477
(Training Classifier) epoch 7; iter: 0; batch classifier loss: 0.672788
(Training Classifier) epoch 8; iter: 0; batch classifier loss: 0.764863
(Training Classifier) epoch 9; iter: 0; batch classifier loss: 0.881486
(Training Classifier) epoch 10; iter: 0; batch classifier loss: 0.853858
(Training Classifier) epoch 11; iter: 0; batch classifier loss: 0.663658
(Training Classifier) epoch 12; iter: 0; batch classifier loss: 0.828861
(Training Classifier) epoch 13; iter: 0; batch classifier

In [8]:
# Apply the plain model to test data
dataset_nodebiasing_train = plain_model.predict(dataset_orig_train)
dataset_nodebiasing_test = plain_model.predict(dataset_orig_test)

In [9]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
metric_dataset_nodebiasing_train = BinaryLabelDatasetMetric(dataset_nodebiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())

metric_dataset_nodebiasing_test = BinaryLabelDatasetMetric(dataset_nodebiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

display(Markdown("#### Plain model - without debiasing - classification metrics"))
classified_metric_nodebiasing_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_nodebiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())

#### Plain model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.213608
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.202229


#### Plain model - without debiasing - classification metrics

Test set: Classification accuracy = 0.803863
Test set: Balanced classification accuracy = 0.655143
Test set: Disparate impact = 0.000000
Test set: Equal opportunity difference = -0.435017
Test set: Average odds difference = -0.268523
Test set: Theil_index = 0.179422


### Apply in-processing algorithm based on adversarial learning


In [10]:
# Learn parameters with debias set to True
debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          debias=True)

In [11]:
debiased_model.fit(dataset_orig_train)

(Pretraining Classifier) epoch 0; iter: 0; batch classifier loss: 134.880569
(Pretraining Classifier) epoch 1; iter: 0; batch classifier loss: 69.873848
(Pretraining Classifier) epoch 2; iter: 0; batch classifier loss: 25.715265
(Pretraining Classifier) epoch 3; iter: 0; batch classifier loss: 1.320486
(Pretraining Classifier) epoch 4; iter: 0; batch classifier loss: 0.858379
(Pretraining Classifier) epoch 5; iter: 0; batch classifier loss: 1.083853
(Pretraining Classifier) epoch 6; iter: 0; batch classifier loss: 0.870830
(Pretraining Classifier) epoch 7; iter: 0; batch classifier loss: 0.941673
(Pretraining Classifier) epoch 8; iter: 0; batch classifier loss: 0.884102
(Pretraining Classifier) epoch 9; iter: 0; batch classifier loss: 0.720615
(Pretraining Classifier) epoch 10; iter: 0; batch classifier loss: 0.705842
(Pretraining Classifier) epoch 11; iter: 0; batch classifier loss: 0.818990
(Pretraining Classifier) epoch 12; iter: 0; batch classifier loss: 0.802354
(Pretraining Class

(Adversarial Debiasing) epoch 34; iter: 0; batch classifier loss: 0.402591; batch adversarial loss: 0.605702
(Adversarial Debiasing) epoch 35; iter: 0; batch classifier loss: 0.413381; batch adversarial loss: 0.613869
(Adversarial Debiasing) epoch 36; iter: 0; batch classifier loss: 0.461322; batch adversarial loss: 0.586686
(Adversarial Debiasing) epoch 37; iter: 0; batch classifier loss: 0.418611; batch adversarial loss: 0.627018
(Adversarial Debiasing) epoch 38; iter: 0; batch classifier loss: 0.446251; batch adversarial loss: 0.576892
(Adversarial Debiasing) epoch 39; iter: 0; batch classifier loss: 0.450661; batch adversarial loss: 0.588187
(Adversarial Debiasing) epoch 40; iter: 0; batch classifier loss: 0.478780; batch adversarial loss: 0.655284
(Adversarial Debiasing) epoch 41; iter: 0; batch classifier loss: 0.448895; batch adversarial loss: 0.660874
(Adversarial Debiasing) epoch 42; iter: 0; batch classifier loss: 0.458956; batch adversarial loss: 0.641847
(Adversarial Debias

In [12]:
# Apply the plain model to test data
dataset_debiasing_train = debiased_model.predict(dataset_orig_train)
dataset_debiasing_test = debiased_model.predict(dataset_orig_test)

In [13]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

# Metrics for the dataset from model with debiasing
display(Markdown("#### Model - with debiasing - dataset metrics"))
metric_dataset_debiasing_train = BinaryLabelDatasetMetric(dataset_debiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_train.mean_difference())

metric_dataset_debiasing_test = BinaryLabelDatasetMetric(dataset_debiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_test.mean_difference())



display(Markdown("#### Plain model - without debiasing - classification metrics"))
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())



display(Markdown("#### Model - with debiasing - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_debiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test.accuracy())
TPR = classified_metric_debiasing_test.true_positive_rate()
TNR = classified_metric_debiasing_test.true_negative_rate()
bal_acc_debiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test.theil_index())

#### Plain model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.213608
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.202229


#### Model - with debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.013427
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.004753


#### Plain model - without debiasing - classification metrics

Test set: Classification accuracy = 0.803863
Test set: Balanced classification accuracy = 0.655143
Test set: Disparate impact = 0.000000
Test set: Equal opportunity difference = -0.435017
Test set: Average odds difference = -0.268523
Test set: Theil_index = 0.179422


#### Model - with debiasing - classification metrics

Test set: Classification accuracy = 0.784003
Test set: Balanced classification accuracy = 0.624387
Test set: Disparate impact = 0.964353
Test set: Equal opportunity difference = 0.127191
Test set: Average odds difference = 0.080554
Test set: Theil_index = 0.196668



    References:
    [1] B. H. Zhang, B. Lemoine, and M. Mitchell, "Mitigating UnwantedBiases with Adversarial Learning," 
    AAAI/ACM Conference on Artificial Intelligence, Ethics, and Society, 2018.